# Domain Generation Algorithm (DGA) Detection

## Authors
 - Gorkem Batmaz (NVIDIA) [gbatmaz@nvidia.com]
 - Bhargav Suryadevara (NVIDIA) [bsuryadevara@nvidia.com]

## Development Notes
* Developed using: RAPIDS v0.12.0 and CLX v0.12
* Last tested using: RAPIDS v0.19.0 and CLX v0.19 on May 5, 2021

## Table of Contents
* Introduction
* Data Importing
* Data Preprocessing
* Training and Evaluation
* Inference
* Conclusion

## Introduction
[Domain Generation Algorithms](https://en.wikipedia.org/wiki/Domain_generation_algorithm) (DGAs) are used to generate domain names that can be used by the malware to communicate with the command and control servers. IP addresses and static domain names can be easily blocked, and a DGA provides an easy method to generate a large number of domain names and rotate through them to circumvent traditional block lists. We will use a type of recurrent neural network called the [Gated Recurrent Unit](https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21) (GRU) for this example. The [CLX](https://github.com/rapidsai/clx) and [RAPIDS](https://rapids.ai) libraries enable users train their models with up-to-date domain names representative of both benign and DGA generated strings. Using a CLX workflow, this capability could also be used in production. This notebook provides a view into the data science workflow to create a DGA detection implementation.

In [1]:
import os
import cudf
import torch
try:
    import s3fs 
except ImportError:
    !conda install -c conda-forge -y s3fs 
    import s3fs 
    
try:
    import clx
except ImportError:
    !conda install -c rapidsai -y clx
    import clx

import numpy as np
from datetime import datetime
from sklearn.metrics import accuracy_score, average_precision_score
from clx.analytics.dga_detector import DGADetector
from clx.utils.data.dataloader import DataLoader
from clx.analytics.dga_dataset import DGADataset
from cuml.preprocessing.model_selection import train_test_split

#### Download Input Dataset from S3

In [2]:
INPUT_CSV = "benign_and_dga_domains.csv"

S3_BASE_PATH = "rapidsai-data/cyber/clx"

In [3]:
# Read Benign and DGA dataset
if not os.path.exists(INPUT_CSV):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + INPUT_CSV, INPUT_CSV)

#### Load Input Dataset to GPU Dataframe

In [4]:
gdf = cudf.read_csv(INPUT_CSV)

In [5]:
train_data = gdf['domain']
labels = gdf['type']

Because we have only benign and DGA (malicious) categoriesm, the number of domain types need to be set to 2 (`N_DOMAIN_TYPE=2`). Vocabulary size(`CHAR_VOCAB`) is set to 128 ASCII characters. The values below set for `HIDDEN_SIZE`, `N_LAYERS` of the network, and the `LR` (Learning Rate) give an optimum balance for the network size and performance. They might need be set via experiments when working with other datasets.

In [6]:
LR = 0.001
N_LAYERS = 3
CHAR_VOCAB = 128
HIDDEN_SIZE = 100
N_DOMAIN_TYPE = 2

#### Instantiate DGA Detector
Now that the data is ready, the datasets are created, and we've set the parameters for the model, we can use the DGADetector method built into CLX to create and train the model.

In [7]:
dd = DGADetector(lr=LR)
dd.init_model(n_layers=N_LAYERS, char_vocab=CHAR_VOCAB, hidden_size=HIDDEN_SIZE, n_domain_type=N_DOMAIN_TYPE)

If you don't have a lot of time, please change EPOCHS to a smaller value.  It takes just under 1 minute per epoch on a P5000 GPU.

In [8]:
EPOCHS = 25
TRAIN_SIZE = 0.7
BATCH_SIZE = 10000
MODELS_DIR = 'models'

### Training and Evaluation
Now we train and evaluate the model.

In [9]:
%%time
dd.train_model(train_data, labels, batch_size=BATCH_SIZE, epochs=EPOCHS, train_size=0.7)

Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

[100000/1433083 (7%)]	Loss: 6844.13
[200000/1433083 (14%)]	Loss: 5165.04
[300000/1433083 (21%)]	Loss: 4015.72
[400000/1433083 (28%)]	Loss: 3421.23
[500000/1433083 (35%)]	Loss: 3195.55
[600000/1433083 (42%)]	Loss: 2775.95
[700000/1433083 (49%)]	Loss: 2448.78
[800000/1433083 (56%)]	Loss: 2198.67
[900000/1433083 (63%)]	Loss: 2215.51
[1000000/1433083 (70%)]	Loss: 2117.47
[1100000/1433083 (77%)]	Loss: 2122.92
[1200000/1433083 (84%)]	Loss: 2052.55
[1300000/1433083 (91%)]	Loss: 2064.78
[1400000/1433083 (98%)]	Loss: 2102.82


Epoch:   4%|▍         | 1/25 [00:55<22:07, 55.31s/it]

Test set: Accuracy: 418521/614179 (0.6814316347514324)

[100000/1433083 (7%)]	Loss: 4007.17
[200000/1433083 (14%)]	Loss: 2857.22
[300000/1433083 (21%)]	Loss: 2123.29
[400000/1433083 (28%)]	Loss: 1786.49
[500000/1433083 (35%)]	Loss: 1567.77
[600000/1433083 (42%)]	Loss: 1348.97
[700000/1433083 (49%)]	Loss: 1184.99
[800000/1433083 (56%)]	Loss: 1060.50
[900000/1433083 (63%)]	Loss: 1023.01
[1000000/1433083 (70%)]	Loss: 1005.30
[1100000/1433083 (77%)]	Loss: 1066.97
[1200000/1433083 (84%)]	Loss: 1054.98
[1300000/1433083 (91%)]	Loss: 1089.22
[1400000/1433083 (98%)]	Loss: 1173.21


Epoch:   8%|▊         | 2/25 [01:49<20:57, 54.68s/it]

...

Test set: Accuracy: 608367/614179 (0.9905369607231768)

[100000/1433083 (7%)]	Loss: 61.71
[200000/1433083 (14%)]	Loss: 44.38
[300000/1433083 (21%)]	Loss: 35.90
[400000/1433083 (28%)]	Loss: 31.01
[500000/1433083 (35%)]	Loss: 29.94
[600000/1433083 (42%)]	Loss: 26.80
[700000/1433083 (49%)]	Loss: 23.95
[800000/1433083 (56%)]	Loss: 22.16
[900000/1433083 (63%)]	Loss: 30.20
[1000000/1433083 (70%)]	Loss: 37.66
[1100000/1433083 (77%)]	Loss: 37.48
[1200000/1433083 (84%)]	Loss: 41.54
[1300000/1433083 (91%)]	Loss: 48.95
[1400000/1433083 (98%)]	Loss: 102.58


Epoch:  96%|█████████▌| 24/25 [21:28<00:53, 53.44s/it]

Test set: Accuracy: 608703/614179 (0.9910840325051817)

[100000/1433083 (7%)]	Loss: 53.78
[200000/1433083 (14%)]	Loss: 37.88
[300000/1433083 (21%)]	Loss: 31.08
[400000/1433083 (28%)]	Loss: 26.78
[500000/1433083 (35%)]	Loss: 42.02
[600000/1433083 (42%)]	Loss: 39.26
[700000/1433083 (49%)]	Loss: 36.59
[800000/1433083 (56%)]	Loss: 34.37
[900000/1433083 (63%)]	Loss: 37.94
[1000000/1433083 (70%)]	Loss: 45.47
[1100000/1433083 (77%)]	Loss: 44.46
[1200000/1433083 (84%)]	Loss: 47.18
[1300000/1433083 (91%)]	Loss: 52.87
[1400000/1433083 (98%)]	Loss: 100.92


Epoch: 100%|██████████| 25/25 [22:22<00:00, 53.70s/it]

Test set: Accuracy: 608702/614179 (0.9910824043153543)

CPU times: user 31min 25s, sys: 1h 27min 9s, total: 1h 58min 34s
Wall time: 22min 26s


### Save Model
Save pretrained model to a given output location.

In [10]:
if not os.path.exists(MODELS_DIR):
    print("Creating directory '{}'".format(MODELS_DIR))
    os.makedirs(MODELS_DIR)

now = datetime.now()
model_filename = "rnn_classifier_{}.bin".format(now.strftime("%Y-%m-%d_%H_%M_%S"))
model_filepath = os.path.join(MODELS_DIR, model_filename)
dd.save_model(model_filepath)
print("Pretrained model saved to location: '{}'".format(model_filepath))

Creating directory 'models'
Pretrained model saved to location: 'models/rnn_classifier_2021-05-05_21_46_27.bin'


### Inference

Using the model generated above, we now score the test dataset against the model to determine if the domain is likely generated by a DGA or benign.

In [11]:
dga_detector = DGADetector()
dga_detector.load_model(model_filepath)

domain_train, domain_test, type_train, type_test = train_test_split(gdf, "type", train_size=0.7)
test_df = cudf.DataFrame()
test_df["type"] = type_test.reset_index(drop=True)
test_df["domain"] = domain_test.reset_index(drop=True)

test_dataset = DGADataset(test_df)
test_dataloader = DataLoader(test_dataset, batchsize=BATCH_SIZE)

pred_results = []
true_results = []
for chunk in test_dataloader.get_chunks():
    pred_results.append(list(dga_detector.predict(chunk['domain']).values_host))
    true_results.append(list(chunk['type'].values_host))
pred_results = np.concatenate(pred_results)
true_results = np.concatenate(true_results)
accuracy_score = accuracy_score(pred_results, true_results)

print('Model accuracy: %s'%(accuracy_score))

Model accuracy: 0.9927285042308512


In [12]:
average_precision = average_precision_score(true_results, pred_results)

print('Average precision score: {0:0.3f}'.format(average_precision))

Average precision score: 0.979


## Conclusion

DGA detector in CLX enables users to train their models for detection and also use existing models. This capability could also be used in conjunction with log parsing efforts if the logs contain domain names. DGA detection done with CLX and RAPIDS keeps data in GPU memory, removing unnecessary copy/converts and providing a 4X speed advantage over CPU only implementations. This is esepcially true with large batch sizes.